In [ ]:
import os 
from dotenv import load_dotenv
import requests
import json
import pandas as pd
load_dotenv()

url = os.environ.get("DQ_URL")
username = os.environ.get("DQ_USERNAME")
password = os.environ.get("DQ_CREDENTIAL")
tenant = os.environ.get("DQ_TENANT")

In [ ]:
def get_api_token():
    prod_auth = requests.post(url + '/v3/auth/signin',
                        headers={'Accept-Language': 'en-US,en;q=0.9', 'Connection': 'keep-alive', 'Content-Type': 'application/json'},
                        data=json.dumps({'username': username, 'password': password, 'iss': tenant}),
                            verify=False)
    token = prod_auth.json()["token"]
    return token

token = get_api_token()
headers = {'Authorization': 'Bearer ' + token}

In [ ]:
connections_list = requests.request('GET', url + '/v2/getconnectionaliases', headers=headers, verify=False)
if connections_list.status_code == 200:
    print(connections_list.json())

In [ ]:
known_list = ['APPROVED_SNOWFLAKE_PUSHDOWN', 'APPROVED_SQL_SERVER_PD', 'APPROVED_SAPHANA_PD', 'APPROVED_BIGQUERY_PUSHDOWN']
for c in connections_list.json():
    if c['isPushdown'] > 0 and c['connectionType'] == 'jdbc' and c['aliasname'] in known_list:
        print(c['aliasname'])

In [ ]:
final_df = pd.DataFrame()

for c in known_list:
    dynamic_tree = requests.request('GET', url + f'/v2/getconnectionschematreebyaliasname?aliasname={c}&showviews=0&eagerfetch=true', headers=headers, verify=False)
    df = pd.DataFrame([
    {"schema": schema, "table": table}
    for schema, tables in dynamic_tree.json().items()
    for table in tables.values()
    ])

    # add column called 'connection_name'
    # make it the first column 
    df.insert(0, 'connection_name', c)

    # Append to the final dataframe
    final_df = pd.concat([final_df, df], ignore_index=True)

# Write to a csv file
final_df.to_csv('connection_schema.csv', index=False)

#final_df.drop_duplicates().count()